# CSC_52081_EP Project

Advanced Machine Learning and Autonomous Agents Project

## Introduction

Reinforcement Learning (RL) has emerged as a robust framework for training autonomous agents to learn optimal behaviors through environmental interactions. This study utilizes the [`CarRacing-v3`](https://gymnasium.farama.org/environments/box2d/car_racing/) environment from Gymnasium, which presents a challenging control task in a racing scenario.

### Environment

The environment features a high-dimensional observation space, represented by a $96 \times 96$ RGB image capturing the car and track, necessitating the use of deep convolutional neural networks (CNNs) for effective feature extraction.

#### Action Space

The action space in CarRacing-v3 supports both continuous and discrete control modes.

In **continuous mode**, the agent outputs three real-valued commands:

- steering (ranging from $-1$ to $+1$)
- gas
- braking

In **discrete mode**, the action space is simplified to five actions:

- do nothing
- steer left
- steer right
- gas
- brake

This dual action representation enables a comprehensive evaluation of various RL algorithms under different control settings.

#### Reward

The reward structure combines a penalty of $-0.1$ per frame and a reward of $+\frac{1000}{N}$ for each new track tile visited, where $N$ is the total number of tiles. This incentivizes the agent to balance exploration (visiting tiles) with efficiency (minimizing frame usage). For example, completing the race after visiting all $N$ tiles in 732 frames yields a reward of $1000 - 0.1 \times 732 = 926.8$ points.

### Objective

The primary objective of this project is to compare RL policies across discrete and continuous action modalities. For discrete control, methods like **Deep Q-Network** (DQN) and **SARSA** are implemented, while continuous control is explored using approaches such as the **Cross-Entropy Method** (CEM), **Self-Adaptive Evolution Strategy** (SA-ES), and policy gradient techniques like **Proximal Policy Optimization** (PPO) and **Soft Actor-Critic** (SAC). This comparative analysis aims to understand the strengths and limitations of each method in handling complex decision spaces.

The high-dimensional visual inputs in `CarRacing-v3` require effective feature extraction, addressed through a tailored CNN architecture. Transitioning between discrete and continuous action representations also demands careful algorithmic design and parameter tuning to ensure stable learning and convergence. While prior studies have often focused on either discrete or continuous action spaces separately, this work adopts a comparative approach, evaluating different agents within the same environment to assess performance under similar conditions.

At this stage, the work outlines the methodology and anticipated challenges, focusing on designing the CNN-based feature extractor, implementing RL algorithms, and establishing a framework for performance comparison. Preliminary findings are yet to be finalized, but the study is expected to provide insights into applying RL in high-dimensional, real-time control tasks. Limitations include the preliminary nature of experiments and the need for further tuning and validation. Future work will involve extensive empirical evaluations, exploring additional policy gradient methods, and refining the network architecture to better handle the complexities of `CarRacing-v3`.

### GitHub

The project's code is available on [GitHub](https://github.com/tr0fin0/ensta_CSC_52081_EP_project), offering a reproducible framework for future investigations and extensions.

## Installation

### Environment

#### WSL, Linux or MacOS

A `Python Virtual Environment` will be used for this project by run the following on a terminal on the project folder:

```bash
sudo apt install python3.10-venv
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

### Imports

In [11]:
import datetime
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import torch


from IPython.display import Video
from pathlib import Path
from typing import List

In [12]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.ion()

In [13]:
sns.set_context("talk")

In [14]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

### Setup

#### Directories

In [15]:
DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")
DIRECTORY_LOGS = Path(f"{DIRECTORY_OUTPUT}/logs/")

if not DIRECTORY_FIGURES.exists():
    DIRECTORY_FIGURES.mkdir(parents=True)

if not DIRECTORY_MODELS.exists():
    DIRECTORY_MODELS.mkdir(parents=True)

if not DIRECTORY_LOGS.exists():
    DIRECTORY_LOGS.mkdir(parents=True)

print(DIRECTORY_OUTPUT)
print(DIRECTORY_MODELS)
print(DIRECTORY_FIGURES)
print(DIRECTORY_LOGS)

output
output\models
output\images
output\logs


## Demonstration

In [16]:
VIDEO_DEMO = "CSC_52081_EP_demonstration"
(DIRECTORY_FIGURES / f"{VIDEO_DEMO}.mp4").unlink(missing_ok=True)


env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DEMO)


done = False
observation, info = env.reset()

while not done:
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DEMO}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

## Description

only demonstration is right. from below here is only experimental.

### Global Definitions

#### Environment

In [17]:
from SkipFrame import SkipFrame

def get_environment_continuous():
    env_cont = gym.make(
        "CarRacing-v3",
        render_mode="rgb_array",
        continuous=True
    )
    env_cont = SkipFrame(env_cont, skip=4)
    env_cont = gym_wrap.GrayscaleObservation(env_cont)
    env_cont = gym_wrap.ResizeObservation(env_cont, shape=(84, 84))
    env_cont = gym_wrap.FrameStackObservation(env_cont, stack_size=4)
    return env_cont

In [18]:
torch.cuda.is_available()

False

### CEM

In [19]:
from Agent_CEM import Agent_CEM
from CNN import CNN
from plot_reward import plot_reward

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Hiperparâmetros do CEM
POPULATION_SIZE = 50
ELITE_FRAC = 0.2
INITIAL_STD = 0.1
GENERATIONS = 100      # número de gerações
EVAL_EPISODES = 1      # episódios para avaliar cada candidato

env = get_environment_continuous()
state, info = env.reset()

# Para o controle contínuo, a dimensão da ação é 3
action_dim = 3

agent = Agent_CEM(
    state_shape=state.shape,
    action_dim=action_dim,
    device=DEVICE,
    directory_models=DIRECTORY_MODELS,
    directory_logs=DIRECTORY_LOGS,
    CNN=CNN,
    population_size=POPULATION_SIZE,
    elite_frac=ELITE_FRAC,
    initial_std=INITIAL_STD,
    load_state=False
)

# Listas para registro de logs
generations_list = []
best_rewards_list = []
avg_rewards_list = []
sigmas_list = []

def evaluate_policy(agent, env, episodes=1):
    """
    Avalia a política atual do agente no ambiente, retornando a recompensa média.
    """
    rewards = []
    for _ in range(episodes):
        state, info = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = agent.get_action(state)
            state, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            done = terminated or truncated
        rewards.append(total_reward)
    return np.mean(rewards)

for generation in range(1, GENERATIONS+1):
    candidates = []
    rewards = []

    # Gera a população e avalia cada candidato
    for i in range(POPULATION_SIZE):
        candidate_params, _ = agent.sample_candidate()
        from torch.nn.utils import vector_to_parameters
        vector_to_parameters(candidate_params, agent.model.parameters())
        candidate_reward = evaluate_policy(agent, env, episodes=EVAL_EPISODES)
        candidates.append(candidate_params)
        rewards.append(candidate_reward)

    best_reward = np.max(rewards)
    avg_reward = np.mean(rewards)

    # Atualiza a política com os candidatos elite
    agent.update_policy(candidates, rewards)

    generations_list.append(generation)
    best_rewards_list.append(best_reward)
    avg_rewards_list.append(avg_reward)
    sigmas_list.append(agent.sigma)

    print(f"Geração {generation}: Best Reward = {best_reward:.2f}, Avg Reward = {avg_reward:.2f}, Sigma = {agent.sigma:.4f}")

    plot_reward(generation, best_rewards_list, sigmas_list)

    # Salva o modelo a cada 10 gerações
    if generation % 10 == 0:
        agent.save(save_name=f"CEM_gen_{generation}")

# Salva modelo final e escreve logs
agent.save()
agent.write_log(generations_list, best_rewards_list, avg_rewards_list, sigmas_list, log_filename="log_CEM.csv")

env.close()
plt.ioff()
plt.show()

Modelo salvo em output\models\CEM_gen_10.pt


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [20]:
# Fase de avaliação
env = get_environment_continuous()
scores = []
for episode in range(5):
    state, info = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = agent.get_action(state)
        state, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        done = terminated or truncated
    scores.append(total_reward)
    print(f"Episódio de Avaliação {episode}: Score = {total_reward:.2f}")

    env.close()

Episódio de Avaliação 0: Score = -78.06
Episódio de Avaliação 1: Score = -78.12
Episódio de Avaliação 2: Score = -79.80
Episódio de Avaliação 3: Score = -76.97
Episódio de Avaliação 4: Score = -79.38
